<a href="https://colab.research.google.com/github/durbin-164/TensorFlow-Basic-Work/blob/master/Implementing_Basic_Neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install tensorflow==2.0.0-beta1

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
print(tf.__version__)

2.0.0-beta1


In [0]:
def sigmoid(z):
  
  return (1.0/(1.0+tf.exp(-z)))

In [3]:
z = tf.constant([[1],[2],[3]], dtype = tf.float32)
print(sigmoid(z))

tf.Tensor(
[[0.7310586 ]
 [0.880797  ]
 [0.95257413]], shape=(3, 1), dtype=float32)


In [0]:
def initialize_parameters(n_x,n_h,n_y):
  
  W1 = tf.random.normal([n_x,n_h])
  b1 = tf.zeros([1,n_h])
  
  W2 = tf.random.normal([n_h,n_y])
  b2 = tf.zeros([1,n_y])
  
  parameters = {
      
      "W1" : W1,
      "b1" : b1,
      "W2" : W2,
      "b2" : b2
  }
  
  return parameters

In [16]:
parm = initialize_parameters(2,3,1)
print(parm['W1'])


tf.Tensor(
[[-2.065537   -0.6189449  -0.590096  ]
 [-0.6218776  -1.0372332   0.12964697]], shape=(2, 3), dtype=float32)


In [0]:
def forward_prop(X, parameters):
  
  W1 = parameters['W1']
  b1 = parameters['b1']
  W2 = parameters['W2']
  b2 = parameters['b2']
  
  Z1 = tf.matmul(X,W1)+b1
  
  A1 = tf.tanh(Z1)
  
  Z2 = tf.matmul(A1, W2)+b2
  
  A2 = sigmoid(Z2)
  
  cache = {
      
      'A1' : A1,
      'A2' : A2
  }
  
  
  return A2, cache
  
  

In [0]:
def calculate_cost(A2, y):
  
  cost = (-1.0/y.shape[0])*tf.reduce_sum(y*tf.math.log(A2)+(1-y)*tf.math.log(1-A2))
  
  return tf.squeeze(cost)

In [0]:
def backward_prop(X,Y,parameters, cache):
  
  W1 = parameters['W1']
  b1 = parameters['b1']
  W2 = parameters['W2']
  b2 = parameters['b2']
  
  A1 = cache['A1']
  A2 = cache['A2']
  
  m = X.shape[0]
  
  dz2 = A2-Y
  dw2 = (1.0/m)* tf.matmul(tf.transpose(A1),dz2)
  db2 = (1.0/m)* tf.reduce_sum(dz2, axis=0, keepdims = True)
  
  dz1 = tf.matmul(dz2, tf.transpose(W2))*(1-tf.square(A1))
  dw1 = (1.0/m)*tf.matmul(tf.transpose(X),dz1)
  db1 = (1.0/m)*tf.reduce_sum(dz1, axis=0, keepdims = True)
  
  grads={
      'dw1' : dw1,
      'db1' : db1,
      'dw2' : dw2,
      'db2' : db2
  }  
  
  return grads

In [0]:
def update_parameters(parameters, grads, learning_rate):
  
  W1 = parameters['W1']
  b1 = parameters['b1']
  W2 = parameters['W2']
  b2 = parameters['b2']
  
  dw1 = grads['dw1']
  db1 = grads['db1']
  dw2 = grads['dw2']
  db2 = grads['db2']
  
  W1 -= learning_rate*dw1
  b1 -= learning_rate*db1
  W2 -= learning_rate*dw2
  b2 -= learning_rate*db2
  
  
  new_parameters = {
      
      "W1" : W1,
      "b1" : b1,
      "W2" : W2,
      "b2" : b2
  }
  
  return new_parameters
  
  
  

In [0]:
def model(X,Y, learning_rate,iteration):
  
  n_x = X.shape[1]
  n_h = 3
  n_y = Y.shape[1]
  
  parameters = initialize_parameters(n_x,n_h,n_y)
  
  all_cost=[]
  
  for i in range(iteration):
    #print('______start FP_________')
    A2,cache = forward_prop(X,parameters)
    #print('______done FP________')      
    cost = calculate_cost(A2,Y)
    #print('____cost done___')
    grads = backward_prop(X,Y,parameters, cache)
    parameters = update_parameters(parameters, grads, learning_rate)
    
    all_cost.append(cost)
    if(i%1000==0):
      print('Iteration: {} :: Cost: {}'.format(i,cost))
      
  return parameters, all_cost

In [0]:
def predict(X,parameters):
  
  A ,cache= forward_prop(X,parameters)
  #print('____done___')
  predict = A>=0.5
  
  return tf.cast(predict,dtype = tf.float32)
  

In [0]:
def get_accuracy(Y,Y_pred):
  
  accuracy = np.sum(tf.equal(Y,Y_pred))
  
  accuracy = (100.0/Y.shape[0])*accuracy
  
  return accuracy

In [0]:
def makeDataset():
  
  df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Dataset/Admission_Predict_Ver1.1.csv')
  
  df.dropna()
  
  df2 = df.copy()
  df2.drop(df.columns[0], axis=1, inplace=True)
  
  data = df.values 
  min_max_scaler = preprocessing.MinMaxScaler()
  data = min_max_scaler.fit_transform(data)
 
  
  np.random.shuffle(data)
  
  data[:,-1] = data[:,-1]>=.6
  
  X = data[:,:-2]
  Y = data[:,-1]
  X= tf.cast(X,dtype=tf.float32)
  Y = tf.reshape(Y, [Y.shape[0],1])
  Y = tf.cast(Y, dtype=tf.float32)
  
  pk = int(0.9*X.shape[0])
  
  X_train = X[:pk,:]
  Y_train = Y[:pk,:]
  
  X_test = X[pk:,:]
  Y_test = Y[pk:,:]
  
  return X_train, Y_train, X_test, Y_test
  

In [47]:
X_train, Y_train, X_test, Y_test = makeDataset()
n_x = X_train.shape[1]
n_y =Y_train.shape[1]
  
num_of_iteration = 30000
learning_rate = 0.05
  
train_parameters, all_cost = model(X_train,Y_train, learning_rate, num_of_iteration)



Iteration: 0 :: Cost: 0.6874687671661377
Iteration: 1000 :: Cost: 0.40042585134506226
Iteration: 2000 :: Cost: 0.29591673612594604
Iteration: 3000 :: Cost: 0.2806439995765686
Iteration: 4000 :: Cost: 0.2775135636329651
Iteration: 5000 :: Cost: 0.27607226371765137
Iteration: 6000 :: Cost: 0.27512291073799133
Iteration: 7000 :: Cost: 0.27433863282203674
Iteration: 8000 :: Cost: 0.27358707785606384
Iteration: 9000 :: Cost: 0.27279987931251526
Iteration: 10000 :: Cost: 0.2719276249408722
Iteration: 11000 :: Cost: 0.2709243893623352
Iteration: 12000 :: Cost: 0.26974835991859436
Iteration: 13000 :: Cost: 0.26837363839149475
Iteration: 14000 :: Cost: 0.2668091356754303
Iteration: 15000 :: Cost: 0.26511350274086
Iteration: 16000 :: Cost: 0.2633897066116333
Iteration: 17000 :: Cost: 0.26175373792648315
Iteration: 18000 :: Cost: 0.26029422879219055
Iteration: 19000 :: Cost: 0.25905027985572815
Iteration: 20000 :: Cost: 0.2580167055130005
Iteration: 21000 :: Cost: 0.2571648955345154
Iteration: 22

In [48]:
#print(train_parameters['b1'].shape)
Y_pred = predict(X_train, train_parameters)
print('Train Accuracy: {}'.format(get_accuracy(Y_train,Y_pred)))

Y_pred = predict(X_test, train_parameters)
print('Test Accuracy : {}'.format(get_accuracy(Y_test,Y_pred)))



(1, 3)
Train Accuracy: 88.22222222222221
Test Accuracy : 84.0
